In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
from datetime import datetime
import pyodbc

# Configuration de la connexion SQL Server
conn = pyodbc.connect(
    r"Driver={ODBC Driver 17 for SQL Server};"
    r"Server=AYOUB\AYOUBTEST;"
    r"Database=BI;"
    "Trusted_Connection=yes;"
)
cursor = conn.cursor()

def get_date_id(date_value, cursor, conn):
    """
    Inserts a date into dim_date if it doesn't exist and returns the date_id.
    """
    query_check = "SELECT date_id FROM dim_date WHERE date = ?"
    query_insert = """
        INSERT INTO dim_date (date, day, month, year, weekday)
        VALUES (?, ?, ?, ?, ?)
    """
    cursor.execute(query_check, (date_value,))
    result = cursor.fetchone()

    if result:
        return result[0]
    else:
        date_obj = datetime.strptime(date_value, "%Y-%m-%d")
        cursor.execute(query_insert, (date_value, date_obj.day, date_obj.month, date_obj.year, date_obj.strftime("%A")))
        conn.commit()
        return cursor.execute(query_check, (date_value,)).fetchone()[0]

def load_data_to_fact(data_dict, metric_type, cursor, conn):
    """
    Inserts daily or yearly data into fact_metrics table.
    """
    try:
        # Prepare metric_type_id
        cursor.execute("SELECT metric_type_id FROM dim_metric_type WHERE metric_type = ?", (metric_type,))
        metric_type_id = cursor.fetchone()[0]

        # Prepare date_id
        today_date = datetime.now().strftime("%Y-%m-%d")
        date_id = get_date_id(today_date, cursor, conn)

        # Insert into fact_metrics
        query_insert = """
            INSERT INTO fact_metrics (
                timestamp, date_id, metric_type_id, current_world_population, deaths,
                undernourished_people_in_the_world, people_died_of_hunger, deaths_due_to_water_related_diseases,
                people_no_access_safe_drinking_water, communicable_disease_deaths, seasonal_flu_deaths,
                deaths_of_children_under_5, hiv_aids_infected, deaths_due_to_hiv_aids, deaths_due_to_cancer,
                deaths_due_to_malaria, deaths_due_to_smoking, deaths_due_to_alcohol, suicides,
                road_traffic_accident_fatalities
            ) VALUES (GETDATE(), ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """

        values = [
            date_id,
            metric_type_id,
            data_dict.get("Current World Population", [0])[-1],
            data_dict.get("Deaths today", [0])[-1] if metric_type == "Daily" else data_dict.get("Deaths this year", [0])[-1],
            data_dict.get("Undernourished people in the world", [0])[-1],
            data_dict.get("People who died of hunger today", [0])[-1] if metric_type == "Daily" else data_dict.get("People who died of hunger this year", [0])[-1],
            data_dict.get("Deaths caused by water-related diseases today", [0])[-1] if metric_type == "Daily" else data_dict.get("Deaths caused by water-related diseases this year", [0])[-1],
            data_dict.get("People with no access to a safe drinking water source", [0])[-1],
            data_dict.get("Communicable disease deaths today", [0])[-1] if metric_type == "Daily" else data_dict.get("Communicable disease deaths this year", [0])[-1],
            data_dict.get("Seasonal flu deaths today", [0])[-1] if metric_type == "Daily" else data_dict.get("Seasonal flu deaths this year", [0])[-1],
            data_dict.get("Deaths of children under 5 today", [0])[-1] if metric_type == "Daily" else data_dict.get("Deaths of children under 5 this year", [0])[-1],
            data_dict.get("HIV/AIDS infected people", [0])[-1],
            data_dict.get("Deaths caused by HIV/AIDS today", [0])[-1] if metric_type == "Daily" else data_dict.get("Deaths caused by HIV/AIDS this year", [0])[-1],
            data_dict.get("Deaths caused by cancer today", [0])[-1] if metric_type == "Daily" else data_dict.get("Deaths caused by cancer this year", [0])[-1],
            data_dict.get("Deaths caused by malaria today", [0])[-1] if metric_type == "Daily" else data_dict.get("Deaths caused by malaria this year", [0])[-1],
            data_dict.get("Deaths caused by smoking today", [0])[-1] if metric_type == "Daily" else data_dict.get("Deaths caused by smoking this year", [0])[-1],
            data_dict.get("Deaths caused by alcohol today", [0])[-1] if metric_type == "Daily" else data_dict.get("Deaths caused by alcohol this year", [0])[-1],
            data_dict.get("Suicide today", [0])[-1] if metric_type == "Daily" else data_dict.get("Suicide this year", [0])[-1],
            data_dict.get("Road traffic accident fatalities today", [0])[-1] if metric_type == "Daily" else data_dict.get("Road traffic accident fatalities this year", [0])[-1],
        ]

        cursor.execute(query_insert, tuple(values))
        conn.commit()
        print(f"Data successfully inserted for {metric_type}")
    except Exception as e:
        print(f"Error inserting data: {e}")

# Set up ChromeDriver using webdriver-manager
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Open the website
url = "https://www.worldometers.info/"
driver.get(url)

# Wait for the page to load
time.sleep(3)

today_data = {
    "Current World Population": [],
    "Deaths today": [],
    "Undernourished people in the world": [],
    "People who died of hunger today": [],
    "Deaths caused by water-related diseases today": [],
    "People with no access to a safe drinking water source": [],
    "Communicable disease deaths today": [],
    "Seasonal flu deaths today": [],
    "Deaths of children under 5 today": [],
    "HIV/AIDS infected people": [],
    "Deaths caused by HIV/AIDS today": [],
    "Deaths caused by cancer today": [],
    "Deaths caused by malaria today": [],
    "Deaths caused by smoking today": [],
    "Deaths caused by alcohol today": [],
    "Suicide today": [],
    "Road traffic accident fatalities today": []
}
year_data = {
    "Current World Population": [],
    "Deaths this year": [],
    "Undernourished people in the world": [],
    "People who died of hunger this year": [],
    "Deaths caused by water-related diseases this year": [],
    "People with no access to a safe drinking water source": [],
    "Communicable disease deaths this year": [],
    "Seasonal flu deaths this year": [],
    "Deaths of children under 5 this year": [],
    "HIV/AIDS infected people": [],
    "Deaths caused by HIV/AIDS this year": [],
    "Deaths caused by cancer this year": [],
    "Deaths caused by malaria this year": [],
    "Deaths caused by smoking this year": [],
    "Deaths caused by alcohol this year": [],
    "Suicide this year": [],
    "Road traffic accident fatalities this year": []
}

def safe_click(element):
    """
    Safely click on an element to avoid issues with stale elements or unclickable elements.
    """
    try:
        element.click()
    except Exception as e:
        print(f"Click failed: {e}")

def extract_data_for_variables(container_xpath, year_key, today_key):
    try:
        container_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, container_xpath)))
        toggle_button = container_element.find_element(By.XPATH, './/a[@class="settype"]')

        if "this year" not in toggle_button.text.strip().lower():
            safe_click(toggle_button)
            time.sleep(2)  # Wait for content to update
        year_value_element = container_element.find_element(By.XPATH, './/span[@class="rts-counter"]')
        year_value = year_value_element.text.strip().replace(",", "")
        year_data[year_key].append(int(year_value))

        if "today" not in toggle_button.text.strip().lower():
            safe_click(toggle_button)
            time.sleep(2)
        today_value_element = container_element.find_element(By.XPATH, './/span[@class="rts-counter"]')
        today_value = today_value_element.text.strip().replace(",", "")
        today_data[today_key].append(int(today_value))

    except Exception as e:
        print(f"Error extracting data: {e}")

    return 

def extract_data():
    population_value = driver.find_element("class name", "rts-counter").text.strip().replace(",", "")
    year_data["Current World Population"].append(int(population_value))
    today_data["Current World Population"].append(int(population_value))
    # Extract additional data
    deaths_this_year = driver.find_element("xpath", '/html/body/div[3]/div[2]/div[2]/div[1]/div/div[5]/span[1]/span').text.strip().replace(",", "")
    year_data["Deaths this year"].append(int(deaths_this_year))

    deaths_today = driver.find_element("xpath", '/html/body/div[3]/div[2]/div[2]/div[1]/div/div[6]/span[1]/span').text.strip().replace(",", "")
    today_data["Deaths today"].append(int(deaths_today))

    undernourished_people = driver.find_element("xpath", '//*[@id="c29"]/div[1]/span[1]/span').text.strip().replace(",", "")
    today_data["Undernourished people in the world"].append(int(undernourished_people))
    year_data["Undernourished people in the world"].append(int(undernourished_people))

    extract_data_for_variables(container_xpath= '//*[@id="c32"]/div[1]', year_key="People who died of hunger this year", today_key="People who died of hunger today")

    extract_data_for_variables(container_xpath= '//*[@id="c36"]/div[1]', year_key="Deaths caused by water-related diseases this year", today_key="Deaths caused by water-related diseases today")

    People_with_no_access_to_a_safe_drinking_water_source = driver.find_element("xpath", '//*[@id="c37"]/div[1]/span[1]/span').text.strip().replace(",", "")
    today_data["People with no access to a safe drinking water source"].append(int(People_with_no_access_to_a_safe_drinking_water_source))
    year_data["People with no access to a safe drinking water source"].append(int(People_with_no_access_to_a_safe_drinking_water_source))

    extract_data_for_variables(container_xpath='//*[@id="c49"]/div[1]', year_key="Communicable disease deaths this year", today_key="Communicable disease deaths today")

    extract_data_for_variables(container_xpath= '//*[@id="flu"]/div[1]', year_key="Seasonal flu deaths this year", today_key="Seasonal flu deaths today")

    extract_data_for_variables(container_xpath= '//*[@id="c50"]/div[1]', year_key="Deaths of children under 5 this year", today_key="Deaths of children under 5 today")
    
    HIV_AIDS_infected_people = driver.find_element("xpath", '//*[@id="c53"]/div[1]/span[1]/span').text.strip().replace(",", "")
    today_data["HIV/AIDS infected people"].append(int(HIV_AIDS_infected_people))
    year_data["HIV/AIDS infected people"].append(int(HIV_AIDS_infected_people))

    extract_data_for_variables(container_xpath= '//*[@id="c54"]/div[1]', year_key="Deaths caused by HIV/AIDS this year", today_key="Deaths caused by HIV/AIDS today")
    
    extract_data_for_variables(container_xpath= '//*[@id="c55"]/div[1]', year_key="Deaths caused by cancer this year", today_key="Deaths caused by cancer today")
    
    extract_data_for_variables(container_xpath= '//*[@id="c56"]/div[1]', year_key="Deaths caused by malaria this year", today_key="Deaths caused by malaria today")
    
    extract_data_for_variables(container_xpath= '//*[@id="c58"]/div[1]', year_key="Deaths caused by smoking this year", today_key="Deaths caused by smoking today")
    
    extract_data_for_variables(container_xpath= '//*[@id="c59"]/div[1]', year_key="Deaths caused by alcohol this year", today_key="Deaths caused by alcohol today")
    
    extract_data_for_variables(container_xpath= '//*[@id="c60"]/div[1]', year_key="Suicide this year", today_key="Suicide today")
    
    extract_data_for_variables(container_xpath= '//*[@id="c62"]/div[1]', year_key="Road traffic accident fatalities this year", today_key="Road traffic accident fatalities today")
    
    return

try:
    while True:
        extract_data()  # Your function to fetch the data

        # Insert data into fact_metrics
        load_data_to_fact(today_data, "Daily", cursor, conn)
        load_data_to_fact(year_data, "Yearly", cursor, conn)
        
        print("Data successfully loaded into the database.")
        time.sleep(10)  # Wait before the next extraction
except KeyboardInterrupt:
    print("Process interrupted.")
    conn.close()
    driver.quit()

Data successfully inserted for Daily
Data successfully inserted for Yearly
Data successfully loaded into the database.
Data successfully inserted for Daily
Data successfully inserted for Yearly
Data successfully loaded into the database.
Data successfully inserted for Daily
Data successfully inserted for Yearly
Data successfully loaded into the database.
Data successfully inserted for Daily
Data successfully inserted for Yearly
Data successfully loaded into the database.
Data successfully inserted for Daily
Data successfully inserted for Yearly
Data successfully loaded into the database.
Process interrupted.
